## Deliverable 2. Create a Customer Travel Destinations Map.

In [34]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Current Description
0,Cidreira,BR,70.57,62,0,11.86,-30.1811,-50.2056,few clouds
1,Tiksi,RU,19.24,99,100,2.06,71.6872,128.8694,few clouds
2,Carnarvon,AU,68.00,73,100,9.22,-24.8667,113.6333,few clouds
3,Lappeenranta,FI,36.00,48,0,2.30,61.0587,28.1887,few clouds
4,Nikolskoye,RU,41.00,60,20,6.71,59.7035,30.7861,few clouds


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method. QUESTION about Unnamed column 

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
                                       
preferred_cities_df.head(10)

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Lat,Lng,Current Description
8,Karratha,AU,79.47,59,6,5.41,-20.7377,116.8463,few clouds
9,Hilo,US,77.00,65,40,3.44,19.7297,-155.0900,few clouds
12,Tiarei,PF,84.20,65,40,8.05,-17.5333,-149.3333,few clouds
13,Nhulunbuy,AU,75.20,88,40,3.44,-12.2333,136.7667,few clouds
19,Saint-Pierre,RE,75.20,78,0,5.75,-21.3393,55.4781,few clouds
22,Avarua,CK,78.80,78,75,9.22,-21.2078,-159.7750,few clouds
23,Oranjestad,AW,84.20,70,75,14.97,12.5240,-70.0270,few clouds
26,Hithadhoo,MV,84.38,69,81,5.48,-0.6000,73.0833,few clouds
30,Bambous Virieux,MU,75.00,94,20,4.61,-20.3428,57.7575,few clouds
31,Puerto Ayora,EC,82.40,65,40,20.71,-0.7393,-90.3518,few clouds


In [22]:
# 4a. Determine if there are any empty rows. 
preferred_cities_df.count()

City                   170
Country                170
Max Temp               170
Humidity               170
Cloudiness             170
Wind Speed             170
Lat                    170
Lng                    170
Current Description    170
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows. 
cleaned_preferred_cities_df = preferred_cities_df.dropna()
cleaned_preferred_cities_df.count()

City                   170
Country                170
Max Temp               170
Humidity               170
Cloudiness             170
Wind Speed             170
Lat                    170
Lng                    170
Current Description    170
dtype: int64

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates. 
cleaned_preferred_cities_df = cleaned_preferred_cities_df[:50]
hotel_df = cleaned_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Karratha,AU,79.47,few clouds,-20.7377,116.8463,
9,Hilo,US,77.00,few clouds,19.7297,-155.0900,
12,Tiarei,PF,84.20,few clouds,-17.5333,-149.3333,
13,Nhulunbuy,AU,75.20,few clouds,-12.2333,136.7667,
19,Saint-Pierre,RE,75.20,few clouds,-21.3393,55.4781,
22,Avarua,CK,78.80,few clouds,-21.2078,-159.7750,
23,Oranjestad,AW,84.20,few clouds,12.5240,-70.0270,
26,Hithadhoo,MV,84.38,few clouds,-0.6000,73.0833,
30,Bambous Virieux,MU,75.00,few clouds,-20.3428,57.7575,
31,Puerto Ayora,EC,82.40,few clouds,-0.7393,-90.3518,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city. QUESTION about part 2
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
cleaned_hotel_df = hotel_df.dropna()

cleaned_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Karratha,AU,79.47,few clouds,-20.7377,116.8463,ibis Styles Karratha
9,Hilo,US,77.00,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
12,Tiarei,PF,84.20,few clouds,-17.5333,-149.3333,Le Rocher de Tahiti
13,Nhulunbuy,AU,75.20,few clouds,-12.2333,136.7667,East Arnhem Real Estate
19,Saint-Pierre,RE,75.20,few clouds,-21.3393,55.4781,Lindsey Hôtel


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
cleaned_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleaned_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))